<a href="https://colab.research.google.com/github/11bender/alumni-scraping/blob/main/alumni_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraping my school's alumni Data from LinkedIn using a bot 🤖**

Install, import and start a browser

In [ ]:
!pip install selenium

In [ ]:
!pip install beautifulsoup4

Download Google Chrome Drive : https://sites.google.com/a/chromium.org/chromedriver/

In [3]:
import os, random, sys, time 
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np #who knows

In [5]:
browser = webdriver.Chrome('/Users/moncefbender/Desktop/enisasTV/MAP/driver/chromedriver')

In [6]:
browser.get("https://www.linkedin.com/login/")

Create a txt file and sign in to your account (better if it's a premium account)

In [8]:
file = open("/Users/moncefbender/Desktop/enisasTV/MAP/config.txt")
line = file.readlines()
username = line[0]
password = line[1]

In [9]:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

Go to your school page, alumni section


In [10]:
browser.get('https://www.linkedin.com/school/ecole-nationale-superieure-d-informatique-et-d-analyse-des-systemes/people/')

Let's scroll all the page

In [ ]:
rep = 50 #determine the rep enough to scroll all the page
last_height = browser.execute_script("return document.body.scrollHeight")

for i in range(rep):
  browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(5)
  new_height = browser.execute_script("return document.body.scrollHeight")
  if new_height == last_height:
    break
  new_height = last_height

It's soup time

In [ ]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

Scrapping profilesID

In [ ]:
pav = soup.find('div', {'class' : 'artdeco-card pv5 pl5 pr1 mt4'})
all_links = pav.find_all('a', {'class' : 'ember-view link-without-visited-state'})

In [ ]:
profilesID = []
for link in all_links:
  profilesID.append(link.get('href'))

In [ ]:
profilesID

['/in/ziyad-harroud-7579831b4/',
 '/in/yousra-berdai/',
 '/in/hamza-ezzaoui-rahali-521730190/',
 '/in/reda-menaoui/',
 '/in/yassine-rachidi-182502175/',
 '/in/anas-saaoudi-b49676181/',
 '/in/salah-eddine-el-baghdadi/',
 '/in/aya-ouzzine-5770241a5/',
 '/in/leilaerrami/',
 '/in/mohamed-jarni/',
 '/in/reda-djohar-778153aa/',
 '/in/elhadri-ranya/']

Scraping names, companies, locations of each profile, (you can define what to scrape by changing id/class)



In [ ]:
names = []
companies = []
locs = []

for profileID in profilesID:
  fulllink = 'https://www.linkedin.com' + profileID
  browser.get(fulllink)
  time.sleep(8)
  src = browser.page_source
  soup = BeautifulSoup(src, 'lxml')

  #scraping name
  name = soup.find('li', {'class' :  'inline t-24 t-black t-normal break-words'}).get_text().strip()
  names.append(name)

  #scraping locations
  name_div = soup.find_all('h4', {'class': 'pv-entity__location'})
  loc = []
  for i in range(len(name_div)):
    span = name_div[i].find_all('span')
    loc.append(span[1].get_text().strip())
  locs.append(loc)

  #scraping companies
  name_div = soup.find_all('p', {'class': 'pv-entity__secondary-title t-14 t-black t-normal'})
  company = []
  for i in range(len(name_div)):
    company.append(name_div[i].get_text().strip())
  companies.append(company)

Let's create our CSV ✌🏻

In [ ]:
df = pd.DataFrame(list(zip(names, companies, locs)), 
               columns =['name', 'companies', 'locations']) 

In [ ]:
df

,name,companies,locations
0,Ziyad HARROUD,[],[]
1,Yousra BERDAI,"[Airbus\n Internship, OCP SA\n I...","[Toulouse, Occitanie, France, Préfecture de Ca..."
2,Hamza Ezzaoui Rahali,[],[]
3,Reda Menaoui,"[Umanis\n Internship, Admiral Digital C...",[]
4,Yassine RACHIDI,[Banque Centrale Populaire - BCP\n Inte...,"[Préfecture de Casablanca, Morocco, Préfecture..."
5,Anas Saaoudi,[],[]
6,Salah-Eddine EL BAGHDADI,[AKUMEN IA (Valyans & AIOX Labs JV)\n I...,[]
7,Aya Ouzzine,[Bazard Tobacco Company- BTC\n Internship],[Tanger et périphérie]
8,Leila Er-rami,[IEEE Student Branch at ENSIAS],"[Rabat, Morocco]"
9,mohamed jarni,[Agence Nationale des Ports (ANP) \n In...,"[El-Jadida, Casablanca-Settat, Maroc, El-Jadid..."


In [ ]:
df["locations"][1]

['Toulouse, Occitanie, France',
 'Préfecture de Casablanca, Morocco',
 'Préfecture de Casablanca, Morocco']

In [ ]:
df["locations"][1][0]

'Toulouse, Occitanie, France'

In [ ]:
df.to_csv('Desktop/data.csv', index=False)

**Made with ♥️ by Moncef Bender.**